In [1]:
import yaml
import math
from pathlib import Path

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

plt.style.use('fivethirtyeight')
plt.style.use('seaborn-poster')

In [2]:
def read_jina(file_path):
    
    with Path(file_path).open() as file:
        file = file.readlines()

    fwf_format = [len(f)+1 for f in file[1].split()] # split on longest line in file and add back in space(s)
    fwf = [0] + [sum(fwf_format[0:i+1]) for i, f in enumerate(fwf_format)]
    

    l = [[] for i in fwf_format]

    for f in file:
        print("|", end='')
        for i in range(len(fwf)-1):
            l[i].append(f[fwf[i]:fwf[i+1]].replace("\n", ""))
            print(f[fwf[i]:fwf[i+1]].replace("\n", ""), end='|')
        print("")

    dic = {}

    for star_index in range(len(fwf_format)-1):
        if l[star_index+1][2].strip() == '1':
            dic[l[star_index+1][4].split()[0]] = {} 

            column_values = l[star_index+1]
            star = column_values[4].split()[0]

            for key, (index, row) in zip(l[0], enumerate(column_values)):
                if index not in [1, 2]:
                    dic[star][key.strip()] = row.strip()
        
    return dic

In [3]:
h_dic = read_jina("../Data/data.txt")
fe_dic = read_jina("../Data/data_fe.txt")
loge_dic = read_jina("../Data/data_loge.txt")

dics = [h_dic, fe_dic, loge_dic]


|JINA_ID    |2020          |2021               |2046          |2047               |2048          |2049              |
|---------- |------------- |------------------ |------------- |------------------ |------------- |------------------|
|Priority   |2             |1                  |2             |2                  |2             |1                 |
|Ref        |COL06         |COL06              |COL06         |COL06              |FRE08         |FRE08             |
|Name       |HE0107-5240   |HE0107-5240 (3D)   |HE1327-2326   |HE1327-2326 (3D)   |HE1327-2326   |HE1327-2326 (3D)  |
|Fe/H       |-5.44         |-5.67              |-5.71         |-5.95              |-5.71         |-6.01             |
|Li/H       |<0.07         |<-0.11             |<0.45         |<0.37              |<-0.35        |<-0.43            |
|C/H        |-1.62         |-2.64              |              |                   |-1.53         |-2.22             |
|N/H        |-3.0          |-3.98              |        

In [4]:
def update_star_file_dic(star, x_axis):
    
    write_file = Path(f"../Data/{star['Name'].split()[0]}.yml")
                      
    try:
        with write_file.open() as file:
            dic = yaml.load(file, Loader=yaml.FullLoader)
    except:
        dic = {"Abundance": {}}

    dic.update({"Star": star["Name"].split()[0], "Jina_ID": int(star["JINA_ID"]), "Ref": star["Ref"]})
    dic["Abundance"][x_axis] = {}
    
    for element, abu in star.items():
        
        el = element.split("/")
        x_label = x_axis.split("/")[-1]

        if (el[-1] == x_label) or ((len(el) == 1) and (len(el[0]) < 3)):
            if abu is not "":
                if abu[0] == '<':
                    el[0] = el[0]+"(UL)"
                    abu = float(abu.replace("-", "-")[1:])
                else:
                    abu = float(abu.replace("-", "-"))
                
                dic['Abundance'][x_axis][el[0]] = abu
                
    with write_file.open("w") as file:
        yaml.dump(dic, file)
    
    return dic 

In [5]:
def add_error(star, x_axis, error):
    
    write_file = Path(f"../Data/{star['Name'].split()[0]}.yml")
                      
    with write_file.open() as file:
        dic = yaml.load(file, Loader=yaml.FullLoader)
        
    dic["Abundance"][f"{x_axis}_error"] = {el: error for el in dic["Abundance"][x_axis].keys()}
                      
    with write_file.open("w") as file:
        yaml.dump(dic, file)
    
    return dic         

In [6]:
update_star_file_dic(h_dic["HE0107-5240"], "X/H")
update_star_file_dic(h_dic["HE1327-2326"], "X/H")

add_error(h_dic["HE0107-5240"], "X/H", 0.2)
add_error(h_dic["HE1327-2326"], "X/H", 0.2)


update_star_file_dic(fe_dic["HE0107-5240"], "X/Fe")
update_star_file_dic(fe_dic["HE1327-2326"], "X/Fe")

add_error(fe_dic["HE0107-5240"], "X/Fe", 0.2)
add_error(fe_dic["HE1327-2326"], "X/Fe", 0.2)


update_star_file_dic(loge_dic["HE0107-5240"], "Loge")
update_star_file_dic(loge_dic["HE1327-2326"], "Loge")

add_error(loge_dic["HE0107-5240"], "Loge", 0.2)
add_error(loge_dic["HE1327-2326"], "Loge", 0.2)

{'Abundance': {'Loge': {'Al': 1.87,
   'Ba(UL)': -2.39,
   'C': 6.21,
   'Ca': 1.26,
   'Co(UL)': 0.31,
   'Cr(UL)': 0.26,
   'Eu(UL)': -0.8,
   'Fe': 1.49,
   'Li(UL)': 0.62,
   'Mg': 3.54,
   'Mn(UL)': 0.55,
   'N': 6.1,
   'Na': 2.94,
   'Ni': 0.45,
   'O': 6.12,
   'Sc(UL)': -1.8,
   'Sr': -1.87,
   'Ti': -0.15,
   'V(UL)': 1.36,
   'Zn(UL)': 1.65},
  'X/Fe': {'Al': 1.43,
   'Ba(UL)': 1.44,
   'C': 3.79,
   'Ca': 0.93,
   'Co(UL)': 1.33,
   'Cr(UL)': 0.63,
   'Eu(UL)': 4.69,
   'Fe': 0.0,
   'Li(UL)': 5.58,
   'Mg': 1.95,
   'Mn(UL)': 1.13,
   'N': 4.28,
   'Na': 2.71,
   'Ni': 0.24,
   'O': 3.44,
   'Sc(UL)': 1.06,
   'Sr': 1.27,
   'Ti': 0.91,
   'V(UL)': 3.44,
   'Zn(UL)': 3.1},
  'X/Fe_error': {'Al': 0.2,
   'Ba(UL)': 0.2,
   'C': 0.2,
   'Ca': 0.2,
   'Co(UL)': 0.2,
   'Cr(UL)': 0.2,
   'Eu(UL)': 0.2,
   'Fe': 0.2,
   'Li(UL)': 0.2,
   'Mg': 0.2,
   'Mn(UL)': 0.2,
   'N': 0.2,
   'Na': 0.2,
   'Ni': 0.2,
   'O': 0.2,
   'Sc(UL)': 0.2,
   'Sr': 0.2,
   'Ti': 0.2,
   'V(UL)': 0.

In [7]:
stars = ["HE0107-5240", "HE1327-2326"]

abundances = ["X/H", "X/Fe", "Loge"]

for dic in dics:
    for star in stars:
        for abu in abundances:
              print(dic[star], abu)
#             update_star_file_dic(dic[star], abu)
#             add_error(dic[star], abu, 0.2)

{'JINA_ID': '2021', 'Ref': 'COL06', 'Name': 'HE0107-5240 (3D)', 'Fe/H': '-5.67', 'Li/H': '<-0.11', 'C/H': '-2.64', 'N/H': '-3.98', 'O/H': '-3.64', 'Na/H': '-4.49', 'Mg/H': '-5.27', 'Al/H': '<-5.66', 'Si/H': '<-5.06', 'Ca/H': '-5.5', 'Sc/H': '<-4.78', 'Ti/H': '-5.61', 'V/H': '', 'Cr/H': '<-5.28', 'Mn/H': '<-5.35', 'Co/H': '<-4.39', 'Ni/H': '-5.95', 'Zn/H': '<-2.54', 'Sr/H': '<-5.87', 'Ba/H': '<-4.77', 'Eu/H': '<-3.04'} X/H
{'JINA_ID': '2021', 'Ref': 'COL06', 'Name': 'HE0107-5240 (3D)', 'Fe/H': '-5.67', 'Li/H': '<-0.11', 'C/H': '-2.64', 'N/H': '-3.98', 'O/H': '-3.64', 'Na/H': '-4.49', 'Mg/H': '-5.27', 'Al/H': '<-5.66', 'Si/H': '<-5.06', 'Ca/H': '-5.5', 'Sc/H': '<-4.78', 'Ti/H': '-5.61', 'V/H': '', 'Cr/H': '<-5.28', 'Mn/H': '<-5.35', 'Co/H': '<-4.39', 'Ni/H': '-5.95', 'Zn/H': '<-2.54', 'Sr/H': '<-5.87', 'Ba/H': '<-4.77', 'Eu/H': '<-3.04'} X/Fe
{'JINA_ID': '2021', 'Ref': 'COL06', 'Name': 'HE0107-5240 (3D)', 'Fe/H': '-5.67', 'Li/H': '<-0.11', 'C/H': '-2.64', 'N/H': '-3.98', 'O/H': '-3.64', 